In [1]:

import import_ipynb
import abuse_detecting_preprocessing as pp
import abuse_detecting_databuild as b

import pandas as pd

from glob import glob
import os 
from transformers import DistilBertTokenizer

tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')





importing Jupyter notebook from abuse_detecting_preprocessing.ipynb


/opt/homebrew/anaconda3/envs/yongari/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


['ㅇㅡㄴㄹㅡㄹㅈㅏㄹㅇㅣ-ㄱㅣㄴㄷㅏ-ㅎㅏ-', 'ㄴㅡㄴㄹㅗㄹㅇㅢ-ㅅㅣㄴㅇㅣ-ㄷㅏ-'] 

['t1ㅇㅡㄴlckㄹㅡㄹㅈㅏㄹㅇㅣ-ㄱㅣㄴㄷㅏ-ㅎㅏ-', 'fakerㄴㅡㄴㄹㅗㄹㅇㅢ-ㅅㅣㄴㅇㅣ-ㄷㅏ-'] 

importing Jupyter notebook from abuse_detecting_databuild.ipynb


In [2]:
def erase_bot(data):

    # 첫 번째 열의 값이 '@'로 시작하는 행을 필터링하여 봇으로 간주하고 제거
    idxs = data[data.str.startswith('@') == True].index
    data.loc[idxs] = None
    bot_cnt = len(idxs)
    print('data.shape : ', data.shape, 'None 처리한 봇 수 : ', bot_cnt, '\n')
    return data
# erase_bot 함수 사용 예시
# data = erase_bot(data)


In [3]:
def erase_emoticons(data):
# 이모티콘들이 대부분 ::smiling::으로 표시되기에 이 부분을 제거하고 이어 붙이는 작업을 수행.
# ex) ::smiling:: 페이커 잘한다 > 페이커 잘한다 
    c = 0
    l = len(data)
    while c != l:
        c += 1
        try:
            cnt = 0
            if ':' in data[cnt]:
                l = data[cnt].find(':')
                r = data[cnt].rfind(':')
                data[cnt] = data[cnt][:l] + data[cnt][r+1:]
                cnt += 1
        except Exception as e:
            print("An error occurred:", e)
            continue
    return data

In [4]:
def null_finder(data): # 공백만 있는 메세지 제거
    print('공백 제거 전\n', data.isnull().value_counts())
    data = data.dropna()
    print('공백 제거 후 \n', data.isnull().value_counts())
    
    return data

In [5]:
def erase_emoticons(data):
    # 각 문장에 대해 이모티콘을 제거하는 작업을 수행
    cleaned_data = []
    for sentence in data:
        while '::' in sentence:
            start = sentence.find('::')
            end = sentence.find('::', start + 2)
            if end == -1:
                break
            sentence = sentence[:start] + sentence[end + 2:]
        cleaned_data.append(sentence)
    return cleaned_data


In [6]:
r = glob('chatting_datas/labeled_datas/*.csv')
data = pd.DataFrame()
for i in r:
    d = pd.read_csv(i)
    data = pd.concat([data, d], axis = 0)
data = data.reset_index(drop = True)
labels = data['label']
message = list(data['message'])


In [7]:
def tokenize_function(texts):
    return tokenizer(texts, padding=True, truncation=True, return_tensors="tf")

tokenized_texts = tokenize_function(message)

In [8]:
import tensorflow as tf

dataset = tf.data.Dataset.from_tensor_slices((dict(tokenized_texts), labels))
dataset = dataset.shuffle(len(message)).batch(32)


In [9]:
from transformers import TFDistilBertForSequenceClassification

model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=2e-5),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = model.fit(dataset, epochs=3)


Epoch 1/3
Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
 24/221 [==>...........................] - ETA: 33:55 - loss: 0.6305 - accuracy: 0.6419

In [ ]:
# 모델 평가
loss, accuracy = model.evaluate(dataset)
print(f'Loss: {loss}, Accuracy: {accuracy}')

# 새로운 텍스트 예측
new_texts = ["Example text to classify"]
new_tokenized_texts = tokenize_function(new_texts)
predictions = model.predict(dict(new_tokenized_texts))
predicted_labels = tf.argmax(predictions.logits, axis=-1).numpy()
print(predicted_labels)
